In [1]:
# Import necessary modules
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import base64
import pandas as pd
import plotly.express as px
from animal_shelter import AnimalShelter  # Adjust based on your module/class name

# Database credentials
username = "aacuser"
password = "your_safe_password"
db = AnimalShelter(username, password)

# Fetch initial data
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

# Encode the logo image
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Initialize the app
app = JupyterDash("AAC Dashboard")

# App layout
app.layout = html.Div([
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '200px'})),
    html.Center(html.B(html.H1("Grazioso Salvare Dashboard"))),
    html.Center(html.I("Sharadiya Sarkar CS-340 Project 2")),
    html.Hr(),
    html.Div([
        html.Label("Select Rescue Type:"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'ALL'},
                {'label': 'Water Rescue', 'value': 'WATER'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'MOUNTAIN'},
                {'label': 'Disaster/Tracking', 'value': 'DISASTER'}
            ],
            value='ALL',
            inline=True
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_data={'whiteSpace': 'normal', 'height': 'auto'},
        sort_action='native',
        filter_action='native',
        row_selectable='single',
    ),
    html.Br(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Callbacks
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_table(filter_type):
    query = {}
    if filter_type == 'WATER':
        query = {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                 "sex_upon_outcome": "Intact Female",
                 "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
    elif filter_type == 'MOUNTAIN':
        query = {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
    elif filter_type == 'DISASTER':
        query = {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
    
    filtered_df = pd.DataFrame.from_records(db.read(query))
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)
    return filtered_df.to_dict('records')

@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_graphs(view_data):
    if not view_data:
        return "No data to display"
    dff = pd.DataFrame.from_dict(view_data)
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return dcc.Graph(figure=fig)

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    # Check if there is data to display
    if viewData is None or len(viewData) == 0:
        return html.Div("No data available")

    dff = pd.DataFrame.from_dict(viewData)

    # Default row selection when no rows are selected
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    # Ensure that latitude and longitude exist in the dataframe
    if 'location_lat' in dff.columns and 'location_long' in dff.columns:
        lat = float(dff.iloc[row]['location_lat'])
        long = float(dff.iloc[row]['location_long'])
    else:
        lat, long = 30.75, -97.48  # Default to Austin if no coordinates are available

    # Return map with marker based on selected row
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[lat, long], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[lat, long], children=[
                       dl.Tooltip(dff.iloc[row]['breed']),  # Assuming 'breed' is in column
                       dl.Popup([
                           html.H1("Animal Name"),
                           html.P(dff.iloc[row]['name'])  # Assuming 'name' is in column
                       ])
                   ])
               ])
    ]
# Run the app
app.run_server(debug=True)


Dash app running on http://127.0.0.1:9359/
